In [ ]:
from tensorflow.random import set_seed 
set_seed(1411)
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
import matplotlib.pyplot as plt
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import plotly.express as px


##Preprocessing of the data

In [ ]:
# load and process the dataset
def load_dataset(pandas_df):
	# load the dataset as a pandas DataFrame
  pandas_df = pandas_df.drop('date', axis=1)
  # retrieve numpy array
  dataset = pandas_df.values
  X = dataset[:, :-1]
  Y = dataset[:,-1]
	# format all fields as string
  X = X.astype(str)
	# reshape target to be a 2d array
  Y = Y.reshape((len(Y), 1))
  return X, Y
# prepare input data
def prepare_inputs(X_train, X_test):
	ohe = OneHotEncoder(sparse=False)
	ohe.fit(X_train)
	X_train_enc = ohe.transform(X_train)
	X_test_enc = ohe.transform(X_test)
	return X_train_enc, X_test_enc

In [ ]:
traffic_data = files.upload()

Saving dummy_traffic_data.csv to dummy_traffic_data (1).csv


In [ ]:
file_handler = open('dummy_traffic_data.csv','r', encoding='utf-8-sig')
traffic = pd.read_csv(file_handler)

In [ ]:
traffic

,date,hour,source,destination,duration
0,2022-02-22,07:00:00,Nürnberg,Ingolstadt,3143.148095
1,2022-02-22,07:05:00,Nürnberg,Ingolstadt,3098.957593
2,2022-02-22,07:10:00,Nürnberg,Ingolstadt,3067.347150
3,2022-02-22,07:15:00,Nürnberg,Ingolstadt,3127.657786
4,2022-02-22,07:20:00,Nürnberg,Ingolstadt,3118.600806
...,...,...,...,...,...
158395,2021-12-25,17:35:00,Ausburg,Munich,2320.107005
158396,2021-12-25,17:40:00,Ausburg,Munich,2295.627799
158397,2021-12-25,17:45:00,Ausburg,Munich,2217.414351
158398,2021-12-25,17:50:00,Ausburg,Munich,2196.928242


In [ ]:
x,y = load_dataset(traffic)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=5)

x_train_encoded, x_test_encoded = prepare_inputs(x_train,x_test)

x_train_encoded = np.asarray(x_train_encoded).astype('float32')
y_train = np.asarray(y_train).astype('float32')
x_test_encoded = np.asarray(x_train_encoded).astype('float32')
y_test = np.asarray(y_train).astype('float32')

In [ ]:
x_train_encoded

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [ ]:
model = Sequential()
model.add(Dense(32, input_dim=x_train_encoded.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.1))
model.add(Dense(128, input_dim=x_train_encoded.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.25))
model.add(Dense(16, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='linear'))

In [ ]:
model.compile(optimizer='Adam',
              loss='mean_squared_error',
              metrics=['mean_absolute_percentage_error'])

In [ ]:
epoch = 35
batch_size = 512
x_train_encoded = np.asarray(x_train_encoded).astype('float32')
y_train = np.asarray(y_train).astype('float32')

model.fit(x_train_encoded, y_train, epochs=epoch, batch_size = batch_size, validation_data = (x_test_encoded,y_test))

Epoch 1/35
279/279 [==============================] - 5s 11ms/step - loss: 12679329.0000 - mean_absolute_percentage_error: 69.7549 - val_loss: 1723730.7500 - val_mean_absolute_percentage_error: 27.8957
Epoch 2/35
279/279 [==============================] - 2s 8ms/step - loss: 1346386.3750 - mean_absolute_percentage_error: 23.7432 - val_loss: 966462.5625 - val_mean_absolute_percentage_error: 19.4275
Epoch 3/35
279/279 [==============================] - 2s 8ms/step - loss: 1055031.6250 - mean_absolute_percentage_error: 20.5030 - val_loss: 914440.0625 - val_mean_absolute_percentage_error: 19.7973
Epoch 4/35
279/279 [==============================] - 2s 8ms/step - loss: 1041401.9375 - mean_absolute_percentage_error: 20.8114 - val_loss: 913293.1875 - val_mean_absolute_percentage_error: 19.9614
Epoch 5/35
279/279 [==============================] - 3s 9ms/step - loss: 1038514.2500 - mean_absolute_percentage_error: 20.8317 - val_loss: 911482.6250 - val_mean_absolute_percentage_error: 19.9334
Ep

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
px.line(losses[['loss','val_loss']])

In [ ]:
px.line(losses[['mean_absolute_percentage_error','val_mean_absolute_percentage_error']])

In [ ]:
y5 = model.evaluate(x_test_encoded,  y_test, verbose=2)
predictor = model.predict(x_test_encoded,verbose=0)
PredictionDf = pd.DataFrame(predictor)
PredictionDf['actual']=y_test
PredictionDf = PredictionDf.rename(columns={0:'predicted'})
PredictionDf.describe()

4455/4455 - 7s - loss: 65062.3281 - mean_absolute_percentage_error: 4.2137 - 7s/epoch - 2ms/step


,predicted,actual
count,142560.000000,142560.000000
mean,4260.173340,4292.611328
std,1412.482422,1474.114502
min,2190.257812,1876.082397
25%,3172.951904,3185.908569
50%,3912.237305,3935.343018
75%,5258.061523,5365.281006
max,7764.175293,9641.698242


In [ ]:
px.line(PredictionDf.head(250))

In [ ]:
PDifference['Prediction Difference'] = (PredictionDf['predicted']-PredictionDf['actual'])/100
PDifference = PDifference['Prediction Difference'].apply(lambda x: float(x))

px.line(PDifference.head(512))

In [ ]:
PDifference.describe()

count    142560.000000
mean         -1.116158
std           2.671344
min         -22.412247
25%          -2.456500
50%          -0.783281
75%           0.540291
max          19.491205
dtype: float64